# 문서 군집화 - Opinion Review dataset

In [1]:
import pandas as pd
import glob, os 

path = 'OpinosisDataset1.0/topics'
os.path.join(path, '*.data') # Windows - \\, Mac - /

'OpinosisDataset1.0/topics\\*.data'

In [2]:
all_files = glob.glob(os.path.join(path, '*.data'))
all_files[:5]

['OpinosisDataset1.0/topics\\accuracy_garmin_nuvi_255W_gps.txt.data',
 'OpinosisDataset1.0/topics\\bathroom_bestwestern_hotel_sfo.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_amazon_kindle.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_ipod_nano_8gb.txt.data',
 'OpinosisDataset1.0/topics\\battery-life_netbook_1005ha.txt.data']

In [3]:
file = all_files[0]
file

'OpinosisDataset1.0/topics\\accuracy_garmin_nuvi_255W_gps.txt.data'

In [4]:
file.split('\\')[-1].split('.')[0]

'accuracy_garmin_nuvi_255W_gps'

In [5]:
filename_list = []
opinion_text = []
for file in glob.glob(os.path.join(path, '*.data')):
    with open(file, encoding='latin1') as f:
        text = f.read()
    opinion_text.append(text)
    filename = file.split('\\')[-1].split('.')[0]
    filename_list.append(filename)

df = pd.DataFrame({'filename':filename_list, 'opinion':opinion_text})
df.head(3)

,filename,opinion
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n but for the m..."
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and ve..."
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my com...


- Simple tokenizer

In [6]:
from nltk import word_tokenize

def simple_tokenizer(text):              # 글자수가 2개 이하인 토큰은 제거
    return [word for word in word_tokenize(text) if len(word) > 2]

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(tokenizer=simple_tokenizer, stop_words='english',
                        ngram_range=(1,2), min_df=0.05, max_df=0.85)
feature = tvect.fit_transform(df.opinion)

* 군집화

In [8]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, max_iter=10000, random_state=2022)
kmeans.fit(feature)

KMeans(max_iter=10000, n_clusters=5, random_state=2022)

In [9]:
df['cluster'] = kmeans.labels_
df.head()

,filename,opinion,cluster
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n but for the m...",0
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and ve...",2
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my com...,1
3,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\...,1
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh ...",1


In [10]:
df.cluster.value_counts()

2    16
3    13
4    10
0     7
1     5
Name: cluster, dtype: int64

In [11]:
# 3개의 집합으로 군집화 
kmeans = KMeans(n_clusters=3, max_iter=10000, random_state=2022)
kmeans.fit(feature)
df['cluster_label'] = kmeans.labels_
df.cluster.value_counts()


2    16
3    13
4    10
0     7
1     5
Name: cluster, dtype: int64

* 군집별 핵심단어 추출하기

In [12]:
feature.shape

(51, 4154)

In [13]:
cluster_centers = kmeans.cluster_centers_
cluster_centers.shape

(3, 4154)

In [14]:
from cluster import get_cluster_details

feature_names = tvect.get_feature_names()
cluster_details = get_cluster_details(cluster_model=kmeans, cluster_data=df, 
                                        feature_names=feature_names, clusters_num=3, top_n_features=10)

In [15]:
for cluster_num, cluster_detail in cluster_details.items():
    print(f'####### Cluster {cluster_num}')
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명:', cluster_detail['filenames'][:7])
    print('==================================================')


####### Cluster 0
Top features: ['screen', 'battery', 'battery life', 'keyboard', 'kindle', 'life', 'directions', 'size', 'voice', 'speed']
Reviews 파일명: ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps']
####### Cluster 1
Top features: ['hotel', 'service', 'rooms', 'staff', 'room', 'food', 'location', 'clean', 'bathroom', 'parking']
Reviews 파일명: ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'location_bestwestern_hotel_sfo', 'location_holiday_inn_london', 'parking_bestwestern_hotel_sfo']
####### Cluster 2
Top features: ['interior', 'mileage', 'seats', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명: ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_hond

In [16]:
cluster_details

{0: {'cluster': 0,
  'top_features': ['screen',
   'battery',
   'battery life',
   'keyboard',
   'kindle',
   'life',
   'directions',
   'size',
   'voice',
   'speed'],
  'top_features_value': [0.13747445010407186,
   0.12055524102848388,
   0.06453150787174568,
   0.06405202909757723,
   0.061187983881947246,
   0.06003451608903705,
   0.054350874237726034,
   0.051096764858376824,
   0.04832059420230829,
   0.047792614773607696],
  'filenames': ['accuracy_garmin_nuvi_255W_gps',
   'battery-life_amazon_kindle',
   'battery-life_ipod_nano_8gb',
   'battery-life_netbook_1005ha',
   'buttons_amazon_kindle',
   'directions_garmin_nuvi_255W_gps',
   'display_garmin_nuvi_255W_gps',
   'eyesight-issues_amazon_kindle',
   'features_windows7',
   'fonts_amazon_kindle',
   'keyboard_netbook_1005ha',
   'navigation_amazon_kindle',
   'performance_netbook_1005ha',
   'price_amazon_kindle',
   'satellite_garmin_nuvi_255W_gps',
   'screen_garmin_nuvi_255W_gps',
   'screen_ipod_nano_8gb',
   'sc